<a href="https://colab.research.google.com/github/calumrussell/fpl/blob/master/elo_post.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**What is ELO?**

ELO is used to compute player or team strength in competitions. All players are rated at 1500 to start, and we update that rating based on the outcome of matches. The difference in rating between two players converts to win/loss probability.

ELO models have two stages: the expectation stage and the update stage.

The expectation stage computes the result expected given the rating difference. In most implementations, the result is a win probability:

$ E_{a} = \frac{1}{1 + 10^{\left ( R_{b} - R_{a} \right )/400}} $

A rating difference of, for example, +200 for team A implies a win 75% of the time. But what about all the random numbers? 400? 10? ELO is most commonly used in chess, and the mapping between rating and win probability is best described by a logistic function. We will see later that we can modify this stage, and the generalised formula is more simple. For now though: this complexity is mostly due to a specific modelling choice.

The update stage compares the result of our match with the result expected by the rating difference, and adjusts for under/over performance. If the rating for A was +200 and they lost, we would reduce our rating by some amount based on this formula: 

$ R_{a}\left ( t+1 \right ) = R_{a}\left ( t \right ) + K\left ( A_{a}-E{_{a}} \right ) $

The parameter K determines the speed of adjustment. A high K value means that the rating changes quickly when we see under/over performance. K of 32 is commonly used but, for applications outside chess, we could determine this value with optimisation.

**Performance**

As far as ratings systems go, ELO is very effective.

Matrix-based rating systems, for example PageRank, don't work with sparse matrices (i.e. where opponents don't necessarily face each other at least one time) and require repeated meetings. ELO makes no real assumptions around the structure of tournaments, and can actually rate teams between competitions assuming that some subset of teams from each competition meet occassionaly.

Regression-based rating systems often produce results that aren't understandable and embed parametric assumptions into our results. If a player gets injured, it is far easier to think about how this affects a team-level rating, as opposed to thinking about how it affects each parameter in a regression-based model or our dependent variable. 

Empirically, results with ELO often compare well with models that are significantly more complex. As we only require match results, ELO is often a useful benchmark model which we can test against more complex models. One possible reason for this is that ELO is really just a Bayesian-updating model: we have some prior in our team rating, and we just update over time. This embeds a huge amount of information in our rating. The Glicko rating model, which is a pure Bayesian model, is a general case of the ELO.

**Improving performance**

The biggest drawback is the simplicity of the model. We may have more information about team/player performance but the implementation above offers no way to embed that information. To include more information, we need to alter the expectation step:

$ E_{a}= \left ( R_{b} - R_{a} \right ) / p $

Once if we make this change, we can use any variable in our update step. In this post, we use the margin of victory (MOV) or goal difference in football.

In code:

In [ ]:
expected_margin = lambda x_rating, y_rating, p: (y_rating-x_rating)/p
rating_change = lambda k, actual_margin, expected_margin: k*(expected_margin-actual_margin)
new_rating = lambda actual_rating, rating_change: actual_rating + rating_change

To compute the values of p and k, we use optimization. In practice, this can be quite tricky. 

First, we tend to have some expectations around the amount of variance that should be in our rating. We pick the intital value, and we want ratings that reflect differences to some degree but don't massively overstate/understate the relative ability of teams. To be clear, this doesn't impact the empirical performance of the model because we only care about the difference between teams, which stays proportionally the same no matter how we scale. But if we have a football model, it doesn't make sense to have a huge gap between top and bottom teams. Interpretability is a key strength of this method.

Second, this optimization appears to be sensitive to starting values irregardless of method. This appears to relate to the first point: with higher k values, you find minimums at higher p values. But the starting values should be chosen with reference to the distribution of the variables underlying the model. If the distribution is right-skewed, for example, then we would need to account for this skew in our model so our ratings aren't effected. 

One particularly importance source of skew in this model is home advantage (i.e. the goal difference is right-skewed for teams playing at home). In our optimization, we introduce an extra parameter to account for this home advantage. Empirically, this led to improved performance in our model, and more understandable changes in ratings based on goal differences after home advantage.

Training our model on data from the EPL 2014-2016, and testing out-of-sample on the 2017 season showed an improvement in Brier score over a conventional ELO:

|Standard ELO| MOV ELO|
|------------|--------|
|0.223       |0.191   |

**Modifications**

If we want to model the probability of a win and the margin of victory together, we can use the normal ELO equation, and add it into our update step:

$ R_{\left ( t+1 \right ) } = R_{\left ( t \right )} + K_{1} (E_{a} - A_{a}) + K_{2}\left (W_{a} - P_{a} \right ) $

The notation is new but the part that we have added onto our previous MOV model update step is just the normal ELO equation. We don't have to add either, we multiply the two parts of this model to reflect a possible nonlinear relationship between greater win margin and ratings.

Once we start making these modifications, it is important to remember the distribution of our MOV. Ideally, we would want changes in our MOV to be expressed consistently in terms of rating. One standard deviation move in MOV should have, in relative terms, a consistent effect on rating and win probability. We would not, for example, want a team that outperforms on MOV but loses to have a disproportionate change in ratings.

This applies to our loss function too: we need to combine log-loss for the standard ELO part of our model but, in most cases, mean squared error for the MOV part.

Another modification is to model the win probability solely as a function of the MOV:

$ L(x) = 1 / (1 + \alpha ^{-x}) $

$ R_{a}\left ( t+1 \right ) = R_{a}\left ( t \right ) + K \left ( L\left ( \left ( E_{a} - A_{a} \right ) / p_{1} \right ) - L\left (\left ( R_{a} - R_{b} \right ) / p_{2} \right ) \right ) $

In this example, we use the logistic transform to scale the MOV between 0 and 1. Again, we should consider the relative weight applied to changes in margin of victory versus win probability. In particular though, because we are working with a transform, we could see big effects on ratings when our two measures diverge. In the source mentioned at the end of this post, this empirical performance of this method is weak.

A simpler approach, one that I haven't compared to the above modifications empirically, is to regress our ratings on win/loss/draw results, and build a win probability that way. In testing, this appeared to performed well with stable parameter estimates, and doesn't come up with some of the modelling complications of the methods above. We can also use this in a multivariate regression, which is important if we want to model draw probability (we do, in the case of football).

Additionally, it makes sense in team invasion games to model offensive and defensive strength seperately. If we attempted to include both of these strength factors in an ELO model though, we would need to deal with the correlation between these factors somehow - team scores are correlated (i.e. a team that scores a lot tends to concede less) - perhaps by adding another parameter. Instead, it makes sense to acknowledge that the key strengths of a modified ELO model is that it can handle limited information, and provide a good benchmark model. For more complex applications, we can take the main strength of ELO-based models - an informative prior that is updated with new information - and work out how to integrate this with more parameters. For example, a purely Bayesian model, like Glicko, that uses more parameters to build priors could capture most of ELO's strengths.

Source: [This paper](https://www.researchgate.net/publication/339406613_Extension_of_the_Elo_rating_system_to_margin_of_victory) was used extensively in writing this article.